In [ ]:
import json
import os
import sys

sys.path.append('../')
sys.path.append('../../')
sys.path.append('../src')
sys.path.append('../prompts')
sys.path.append('../src/llmperf')

import pandas as pd

# Synthetic - Consolidate results


In [ ]:
def read_json_files(folder_path):
    data = []
    
    # Iterate through all files in the folder
    for filename in os.listdir(folder_path):
        # Check if the file ends with 'individual_responses.json'
        if filename.endswith('individual_responses.json'):
            file_path = os.path.join(folder_path, filename)
            
            # Open and load the JSON file
            with open(file_path, 'r') as file:
                try:
                    json_data = json.load(file)
                    json_data = [{**request_response, 'filename': filename} for request_response in json_data]
                    data.append(json_data)
                except json.JSONDecodeError as e:
                    print(f"Error reading {file_path}: {e}")
    return data

In [ ]:
results_dir = '../data/results/path/'
all_responses = read_json_files(results_dir)
print(f'Number of files found {len(all_responses[0])}')

In [ ]:
run_stats = []
for run in all_responses:
    metrics = {
        'filename': [],
        'input_tokens': [],
        'output_tokens': [],
        'concurrent_requests': [], 
        'server_number_input_tokens': [],
        'server_number_output_tokens': [],
        'server_ttft_s': [], 
        'server_output_token_per_s_per_request': [],
        'server_end_to_end_latency_s': [],
        'client_ttft_s': [], 
        'client_output_token_per_s_per_request': [], 
        'client_end_to_end_latency_s': []
    }
    for request_metrics in run:
        # It will skip the requests that have an error message
        if len(request_metrics['error_msg'])==0:
            metrics['filename'].append(request_metrics['filename'])
            input_tokens = int(request_metrics['filename'].split('_')[3])
            output_tokens = int(request_metrics['filename'].split('_')[4])
            concurrent_requests = int(request_metrics['filename'].split('_')[5])
            metrics['input_tokens'].append(input_tokens)
            metrics['output_tokens'].append(output_tokens)
            metrics['concurrent_requests'].append(concurrent_requests)
            
            metrics['server_number_input_tokens'].append(request_metrics['server_number_input_tokens'])
            metrics['server_number_output_tokens'].append(request_metrics['server_number_output_tokens'])
            metrics['server_ttft_s'].append(request_metrics['server_ttft_s'])
            metrics['server_output_token_per_s_per_request'].append(request_metrics['server_output_token_per_s_per_request'])
            metrics['server_end_to_end_latency_s'].append(request_metrics['server_end_to_end_latency_s'])
            metrics['client_ttft_s'].append(request_metrics['client_ttft_s'])
            metrics['client_output_token_per_s_per_request'].append(request_metrics['client_output_token_per_s_per_request'])
            metrics['client_end_to_end_latency_s'].append(request_metrics['client_end_to_end_latency_s'])
    df_metrics =  pd.DataFrame(metrics)
    df_metric_stats = df_metrics.groupby(by='filename')[['server_ttft_s','server_output_token_per_s_per_request','server_end_to_end_latency_s','client_ttft_s','client_output_token_per_s_per_request','client_end_to_end_latency_s']].agg(['median','std'])
    df_metric_stats.style.format("{:,.3f}")
    df_parameters = df_metrics.groupby(by='filename')[['input_tokens','output_tokens','concurrent_requests']].agg(['first'])    
    df_parameters['request_count'] = df_metrics.shape[0]
    df_parameters.style.format("{:,.0f}")
    df = pd.concat([df_parameters, df_metric_stats], axis=1)
    run_stats.append(df)
results = pd.concat(run_stats)
results.columns = ['_'.join(col).strip() for col in results.columns.values]
results.columns = [col.replace('_first','') for col in results.columns.values]
results = results.sort_values(by=['input_tokens','output_tokens','concurrent_requests'])
results